## ML reproduction

In [ ]:
import sys
sys.modules

In [ ]:
!pip install numpy seaborn matplotlib pandas joblib ipdb scikit-learn==0.23

In [2]:
import sys, os, fnmatch, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from joblib import dump, load

from sklearn.model_selection import train_test_split

#sys.path.insert(0, os.path.dirname(os.getcwd()))

PATH_MODELS = './models'
PATH_DATA_PROCESSED_ML =  './data_prepro_ml'

In [3]:

file_names = os.listdir(PATH_DATA_PROCESSED_ML)
file_names

['processed_data_008_11_jc_mmn_36_slp_mmn25_slp_mmn47_wk_2.csv',
 'processed_data_006_29_mc_mmn36_1_wk.csv',
 'processed_data_006_23_mc_mmn25_slp.csv',
 'extracted_features_004_29_mc_mmn25_2_wk.h5',
 'extracted_features_004_23_mc_mmn36_wk.h5',
 'processed_data_007_17_jc_mmn2.csv',
 'processed_data_001_23_jc_mmn36_wk_mmn25_wk.csv',
 'processed_data_005_35_jc_mmn36_wk.csv',
 'extracted_features_005_29_jc_mmn25.h5',
 'extracted_features_1_nakijken_mmn3-6.h5',
 'processed_data_007_23_jc_mmn36_wk.csv',
 'processed_data_008_11_jc_mmn_3.csv',
 'processed_data_001_29_jc_mmn.csv',
 'extracted_features_008_11_jc_mmn_3.h5',
 'processed_data_002_23_jc_mmn36_wk_mmn25.csv',
 'processed_data_009_29_jc_mmn36_wk.csv',
 'extracted_features_005_29_jc_mmn36_wk.h5',
 'processed_data_009_23_jc_mmn25_wk.csv',
 'extracted_features_007_23_jc_mmn25_wk.h5',
 'processed_data_009_17_jc_mmn36_wk_mmn25_wk.csv',
 'processed_data_009_23_jc_mmn36_wk.csv',
 'extracted_features_002_29_jc_mmn25_wk.h5',
 'processed_data_1_

In [4]:
!pip install tables

In [5]:
%%time

from sklearn.model_selection import train_test_split

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_DATA_PROCESSED_ML)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(os.path.join(PATH_DATA_PROCESSED_ML, file_name))[0] for file_name in fnmatch.filter(file_names, "*.h5")]

# Step 3: Load the features
frames = []

for idx, feature_file in enumerate(files):
    df_features = pd.read_hdf(feature_file + ".h5")
    df_metadata = pd.read_csv(feature_file.replace("extracted_features_", "processed_data_") + ".csv")
    
    # Step 4: Assign labels
    df_features['label'] = df_metadata['age_months'][0]
    
    # Step 5: Assign subject code
    df_features['code'] = df_metadata['code'][0]
    frames.append(df_features)

df = pd.concat(frames) 

# Step 6: List all the unique subject IDs
subject_ids = sorted(list(set(df["code"].tolist())))

IDs_train, IDs_temp = train_test_split(subject_ids, test_size=0.3, random_state=42)
IDs_test, IDs_val = train_test_split(IDs_temp, test_size=0.5, random_state=42)

# Step 7: Split the DataFrames into train, validation and test
df_train = df[df['code'].isin(IDs_train)]
df_val = df[df['code'].isin(IDs_val)]
df_test = df[df['code'].isin(IDs_test)]

feature_names = df.columns.values

CPU times: user 582 ms, sys: 120 ms, total: 702 ms
Wall time: 708 ms


In [6]:
print(f"Train/val/test proportions: {len(df_train)/len(df)}/{len(df_val)/len(df)}/{len(df_test)/len(df)}")

Train/val/test proportions: 0.5981548335338949/0.2833935018050541/0.11845166466105095


In [7]:
%%time

X_train = df_train.drop(['label', 'code'], axis=1).reset_index(drop=True)
y_train = df_train['label'].reset_index(drop=True)
codes_train = df_train['code'].reset_index(drop=True)

X_val = df_val.drop(['label', 'code'], axis=1).reset_index(drop=True)
y_val = df_val['label'].reset_index(drop=True)
codes_val = df_val['code'].reset_index(drop=True)

X_test = df_test.drop(['label', 'code'], axis=1).reset_index(drop=True)
y_test = df_test['label'].reset_index(drop=True)
codes_test = df_test['code'].reset_index(drop=True)

from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.fit_transform(X_val)
# X_test = scaler.fit_transform(X_test)

# MARK: reducing from 64 bit float to 32 bit float, to reduce memory usage
X_train = pd.DataFrame(scaler.fit_transform(X_train)).astype('float32')
X_val = pd.DataFrame(scaler.fit_transform(X_val)).astype('float32')
X_test = pd.DataFrame(scaler.fit_transform(X_test)).astype('float32')

/home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:981: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:992: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count
/home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:1012: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count
/home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:981: RuntimeWarning: invalid value encountered in true_divide
  updated_mean = (last_sum + new_sum) / updated_sample_count
/home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:992: RuntimeWarning: invalid value encountered in true_divide
  T = new_sum / new_sample_count


CPU times: user 338 ms, sys: 94.6 ms, total: 432 ms
Wall time: 440 ms


/home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:1012: RuntimeWarning: invalid value encountered in true_divide
  new_unnormalized_variance -= correction**2 / new_sample_count


In [8]:
X_train_val = pd.concat([X_train, X_val])
y_train_val = pd.concat([y_train, y_val])
codes_train_val = pd.concat([codes_train, codes_val])

In [9]:
del(file_names, files, df, frames, df_features, df_metadata, df_train, df_test, df_val, IDs_train, IDs_val, IDs_test, IDs_temp)

In [10]:
print(f"{X_train.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_train.memory_usage(deep=True)/1000000000}")
print(f"{codes_train.memory_usage(deep=True)/1000000000}")
print(f"{X_val.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_val.memory_usage(deep=True)/1000000000}")
print(f"{codes_val.memory_usage(deep=True)/1000000000}")
print(f"{X_train_val.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_train_val.memory_usage(deep=True)/1000000000}")
print(f"{codes_train_val.memory_usage(deep=True)/1000000000}")
print(f"{X_test.memory_usage(deep=True).sum()/1000000000}")
print(f"{y_test.memory_usage(deep=True)/1000000000}")
print(f"{codes_test.memory_usage(deep=True)/1000000000}")

0.053683328
0.000119424
0.000119424
0.025434128
5.6648e-05
5.6648e-05
0.079293016
0.000351632
0.000351632
0.010630928
2.3752e-05
2.3752e-05


In [11]:
del(X_train, y_train, codes_train, X_val, y_val, codes_val)

In [12]:
from sklearn.utils import shuffle

# Shuffle data before using
X_train_val, y_train_val, codes_train_val = shuffle(X_train_val, y_train_val, codes_train_val, random_state=42)

chunked_X_train = np.array_split(X_train_val, 100)    
chunked_y_train = np.array_split(y_train_val, 100)

In [13]:
chunks_X_train_ten = []
chunks_y_train_ten = []

for i in range(10):
    chunks_X_train_ten.append(chunked_X_train[i])
    chunks_y_train_ten.append(chunked_y_train[i])

chunks_X_train_ten = pd.concat(chunks_X_train_ten)
chunks_y_train_ten = pd.concat(chunks_y_train_ten)

In [14]:
%%time

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

dummy_regr = DummyRegressor(strategy="mean")
dummy_regr.fit(X_train_val, y_train_val)

output_file = os.path.join(PATH_MODELS, 'Dummy_mean.joblib')
dump(dummy_regr, output_file)

CPU times: user 1.28 ms, sys: 0 ns, total: 1.28 ms
Wall time: 1.41 ms


['./models/Dummy_mean.joblib']

In [15]:
%%time

from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error

dummy_regr = DummyRegressor(strategy="median")
dummy_regr.fit(X_train_val, y_train_val)

output_file = os.path.join(PATH_MODELS, 'Dummy_median.joblib')
dump(dummy_regr, output_file)

CPU times: user 1.14 ms, sys: 0 ns, total: 1.14 ms
Wall time: 1.58 ms


['./models/Dummy_median.joblib']

In [16]:
# %%time

# from sklearn.ensemble import RandomForestRegressor
# from sklearn.metrics import mean_squared_error
# from sklearn.model_selection import GridSearchCV

# # NOTE: Gridsearch was extended multiple times initial search, therefore the results 
# # for max_features = 40, 60, 70, 80, 90, 100, 150, 250, None can be found in 
# # "RF_gridsearchX.joblib" files (X = 2, 3, 4, 5)

# parameters = { 
#               'max_features': ['sqrt', 'log2', 15, 30, 40, 50, 60, 70, 80, 90, 100, 150, 250, None],
#               'min_samples_leaf': [1, 2, 3, 4, 5, 10, 20, 30, 40, 50]
#              }

# RF_gridsearch = GridSearchCV(RandomForestRegressor(n_estimators=100, verbose=10, n_jobs=-1), 
#                              parameters, verbose=10, n_jobs=1)

# RF_gridsearch.fit(X_train_val, y_train_val)

# output_file = os.path.join(PATH_MODELS, 'RF_gridsearch5.joblib')
# dump(RF_gridsearch, output_file)

In [17]:
RF_gridsearch.best_params_

NameError: name 'RF_gridsearch' is not defined

In [ ]:
%%time

import importlib

# importlib.reload(np)
np.seterr(all='raise')

from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import make_scorer, mean_absolute_error
from scipy.stats import uniform

# Started with large parameter space, decreased the range of parameters and increased training data
# RS with 1/10th of data shows that C = 0.75 performs best and epsilon = 2.5
parameters = {'linearsvr__C': uniform(0.01, 1),
              'linearsvr__epsilon': uniform(0, 6),
}

scorer = make_scorer(mean_absolute_error, greater_is_better=False)

pipe  = make_pipeline(StandardScaler(),
                      LinearSVR(verbose=2, max_iter=50000))

LSVR_randomsearch = RandomizedSearchCV(pipe, parameters, n_iter=100, cv=5, n_jobs=-1, scoring=scorer, verbose=10)
# LSVR_randomsearch.fit(X_train_val, y_train_val)
LSVR_randomsearch.fit(chunks_X_train_ten, chunks_y_train_ten)

output_file = os.path.join(PATH_MODELS, 'LSVR_randomsearch.joblib')
dump(LSVR_randomsearch, output_file)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV 1/5; 1/100] START linearsvr__C=0.834745097290563, linearsvr__epsilon=5.127483195687349
> /home/cmoore/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py(985)_incremental_mean_and_var()
    983         from IPython.core.debugger import set_trace
    984         set_trace()
--> 985         print('updated_sample_count', updated_sample_count)
    986         print('sum', last_sum + new_sum)
    987         raise



ipdb>  updated_sample_count


array([1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760, 1760,
       1760, 1760, 1760, 1760, 1760, 1760, 1760, 17

ipdb>  last_sum


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

...

1760


ipdb>  X


array([[-0.4297868 , -0.3064694 , -0.3860518 , ...,         nan,
                nan,         nan],
       [ 1.4344859 ,  1.644264  ,  1.7644737 , ...,         nan,
                nan,         nan],
       [-0.25884303, -1.0847396 , -1.0948586 , ...,         nan,
                nan,         nan],
       ...,
       [ 0.01262396,  1.1252204 ,  0.5714156 , ...,         nan,
                nan,         nan],
       [ 0.1942347 , -0.03161406,  0.12647119, ...,         nan,
                nan,         nan],
       [-0.37361547, -0.6177594 , -0.52353966, ...,         nan,
                nan,         nan]], dtype=float32)


ipdb>  bt


    [... skipping 21 hidden frame(s)]

  /tmp/ipykernel_32337/1925296218.py(1)<cell line: 1>()
----> 1 get_ipython().run_cell_magic('time', '', "\nimport importlib\n\n# importlib.reload(np)\nnp.seterr(all='raise')\n\nfrom sklearn.model_selection import RandomizedSearchCV\nfrom sklearn.svm import LinearSVR\nfrom sklearn.preprocessing import StandardScaler\nfrom sklearn.pipeline import make_pipeline\nfrom sklearn.metrics import make_scorer, mean_absolute_error\nfrom scipy.stats import uniform\n\n# Started with large parameter space, decreased the range of parameters and increased training data\n# RS with 1/10th of data shows that C = 0.75 performs best and epsilon = 2.5\nparameters = {'linearsvr__C': uniform(0.01, 1),\n              'linearsvr__epsilon': uniform(0, 6),\n}\n\nscorer = make_scorer(mean_absolute_error, greater_is_better=False)\n\npipe  = make_pipeline(StandardScaler(),\n                      LinearSVR(verbose=2, max_iter=50000))\n\nLSVR_randomsearch = RandomizedSearchCV(pip

In [ ]:
LSVR_randomsearch?

In [ ]:
LSVR_randomsearch

In [ ]:
cv_DF = pd.DataFrame(LSVR_randomsearch.cv_results_).sort_values('rank_test_score').head(50)

pd.DataFrame(LSVR_randomsearch.cv_results_).sort_values('rank_test_score').head(30)

# Best C: 1.04, epsilon: 1.467

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set()
ax = sns.scatterplot(x="param_linearsvr__C", y="param_linearsvr__epsilon", hue="mean_test_score", palette='RdBu', data=cv_DF)

norm = plt.Normalize(cv_DF['mean_test_score'].min(), cv_DF['mean_test_score'].max())
sm = plt.cm.ScalarMappable(cmap="RdBu", norm=norm)
sm.set_array([])

# Remove the legend and add a colorbar
ax.get_legend().remove()
ax.figure.colorbar(sm)

plt.show()

In [ ]:
%%time

from sklearn.svm import LinearSVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_absolute_error

parameters = {'linearsvr__C': [0.4, 0.45, 0.5, 0.55, 0.6, 0.65, 0.7, 0.75, 0.8],
              'linearsvr__epsilon': [1.5, 2, 2.5, 3],
}

scorer = make_scorer(mean_absolute_error, greater_is_better=False)

pipe  = make_pipeline(StandardScaler(),
                      LinearSVR(verbose=2, max_iter=50000))

LSVR_gridsearch = GridSearchCV(pipe, parameters, cv=5, n_jobs=-1,  scoring=scorer, verbose=10)
# LSVR_gridsearch.fit(X_train_val, y_train_val)
LSVR_gridsearch.fit(chunks_X_train_ten, chunks_y_train_ten)

output_file = os.path.join(PATH_MODELS, 'LSVR_gridsearch.joblib')
dump(LSVR_gridsearch, output_file)

In [ ]:
!python --version